In [1]:
# !pip install ffmpeg-python

Collecting ffmpeg-python

  Downloading ffmpeg_python-0.2.0-py3-none-any.whl (25 kB)
  
Requirement already satisfied: future in c:\programdata\anaconda3\lib\site-packages (from ffmpeg-python) (0.18.2)

Installing collected packages: ffmpeg-python

Successfully installed ffmpeg-python-0.2.0

In [6]:
import ffmpeg
import os
import subprocess

In [22]:
source = "TestFile.mp3"
output = "OutFile.mp3"
check = "TestFile2.mp3"

In [4]:
spd = 1.5 # output speed
vol = 1.0 # volume rise ratio if any
bit = 16 # BitRate: 16K
frq = 24000 # sampling stream frequency

#Temporary parameters
src, out = source, output
# commands
stg1 = f'ffmpeg -i {src} -filter:a "atempo={spd},volume={vol}" -q:a 100 {out}'
stg2 = f"ffmpeg -i {src} -ac 1 -b:a {bit}k -ar {frq} -write_xing 0 {output}"

### Finding silence and intervals, from this post
https://stackoverflow.com/questions/14590279/error-nameerror-name-subprocess-is-not-defined

In [12]:
command = f"ffmpeg -i {src} -af silencedetect=n=-30dB:d=0.25 -f null - "
out = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)

stdout, stderr = out.communicate()
stdout

b"ffmpeg version 4.3.1 Copyright (c) 2000-2020 the FFmpeg developers\r\n  built with gcc 10.2.1 (GCC) 20200726\r\n  configuration: --disable-static --enable-shared --enable-gpl --enable-version3 --enable-sdl2 --enable-fontconfig --enable-gnutls --enable-iconv --enable-libass --enable-libdav1d --enable-libbluray --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopus --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libsrt --enable-libtheora --enable-libtwolame --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libzimg --enable-lzma --enable-zlib --enable-gmp --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvo-amrwbenc --enable-libmysofa --enable-libspeex --enable-libxvid --enable-libaom --enable-libgsm --enable-librav1e --disable-w32threads --enable-libmfx --enable-ffnvcodec --enable-cuda-llvm --enable-cuvid --enable-

In [21]:
splits = str(stdout).split("100 pcm_s16le\\r\\n")[1].split("\\r\\nsize=")[0]
splits = splits.split("\\r\\n")
splits

['[silencedetect @ 0000021c52e867c0] silence_start: 14.1607',
 '[silencedetect @ 0000021c52e867c0] silence_end: 14.5508 | silence_duration: 0.390113',
 '[silencedetect @ 0000021c52e867c0] silence_start: 14.5509',
 '[silencedetect @ 0000021c52e867c0] silence_end: 14.8773 | silence_duration: 0.32644',
 '[silencedetect @ 0000021c52e867c0] silence_start: 53.0442',
 '[silencedetect @ 0000021c52e867c0] silence_end: 53.3589 | silence_duration: 0.314671',
 '[silencedetect @ 0000021c52e867c0] silence_start: 67.8339',
 '[silencedetect @ 0000021c52e867c0] silence_end: 68.1681 | silence_duration: 0.334218',
 '[silencedetect @ 0000021c52e867c0] silence_start: 114.954',
 '[silencedetect @ 0000021c52e867c0] silence_end: 115.233 | silence_duration: 0.279206',
 '[silencedetect @ 0000021c52e867c0] silence_start: 131.051',
 '[silencedetect @ 0000021c52e867c0] silence_end: 131.385 | silence_duration: 0.333991',
 '[silencedetect @ 0000021c52e867c0] silence_start: 143.995',
 '[silencedetect @ 0000021c52e867

#### Let's apply the same to other file, to see the output format

In [23]:
def get_silence_times(src):
    command = f"ffmpeg -i {src} -af silencedetect=n=-30dB:d=0.25 -f null - "
    out = subprocess.Popen(command,
                           stdout=subprocess.PIPE,
                           stderr=subprocess.STDOUT)
    stdout, stderr = out.communicate()
    splits = str(stdout).split("100 pcm_s16le\\r\\n")[1].split(
        "\\r\\nsize=")[0]
    splits = splits.split("\\r\\n")
    return splits

In [24]:
get_silence_times(check)

['[silencedetect @ 000001e510f2b080] silence_start: 0',
 '[silencedetect @ 000001e510f2b080] silence_end: 0.766417 | silence_duration: 0.766417',
 '[silencedetect @ 000001e510f2b080] silence_start: 3.41925',
 '[silencedetect @ 000001e510f2b080] silence_end: 5.25726 | silence_duration: 1.838',
 '[silencedetect @ 000001e510f2b080] silence_start: 6.18304',
 '[silencedetect @ 000001e510f2b080] silence_end: 6.96218 | silence_duration: 0.779138',
 '[silencedetect @ 000001e510f2b080] silence_start: 9.79152',
 '[silencedetect @ 000001e510f2b080] silence_end: 10.9414 | silence_duration: 1.14986',
 '[silencedetect @ 000001e510f2b080] silence_start: 11.2242',
 '[silencedetect @ 000001e510f2b080] silence_end: 12.2511 | silence_duration: 1.02696',
 '[silencedetect @ 000001e510f2b080] silence_start: 17.3094',
 '[silencedetect @ 000001e510f2b080] silence_end: 18.0719 | silence_duration: 0.762426',
 '[silencedetect @ 000001e510f2b080] silence_start: 18.7098',
 '[silencedetect @ 000001e510f2b080] silen

It appears that there are 3 lines at the end of outputs that are not related to `silencedetect` 
